# Using `qdk-chemistry` for multi-reference quantum chemistry state preparation and iterative quantum phase estimation

This notebook demonstrates an end-to-end multi-configurational quantum chemistry workflow using `qdk-chemistry`.

It covers molecule loading and visualization, self-consistent-field (SCF) calculation, active-space selection, multi-configurational wavefunction generation, quantum state-preparation circuit construction, and Iterative Quantum Phase Estimation (IQPE) for ground-state energy estimation.

## Loading the molecular structure

For this example, we will use a stretched H2 molecule to demonstrate the end-to-end workflow.

In [28]:
import numpy as np

from qdk_chemistry.data import Structure

# Streched H2 structure at 2.5 Bohr
structure = Structure(np.array([[0, 0, 0], [0, 0, 2.5]]), symbols=["H", "H"])

## Generating the molecular orbitals

This step performs a Hartree-Fock (HF) SCF calculation to generate an approximate initial wavefunction and ground-state energy guess. The resulting molecular orbitals will be used in subsequent steps for active space selection and multi-configuration calculations.

In [29]:
from qdk_chemistry.algorithms import create

# Perform an SCF calculation, returning the energy and wavefunction
scf_solver = create("scf_solver", basis_set="cc-pvdz")
E_hf, wfn_hf = scf_solver.run(structure, charge=0, spin_multiplicity=1)
print(f"SCF energy is {E_hf:.3f} Hartree")

# Display a summary of the molecular orbitals obtained from the SCF calculation
print("SCF Orbitals:\n", wfn_hf.get_orbitals().get_summary())

SCF energy is -1.036 Hartree
SCF Orbitals:
 Orbitals Summary:
  AOs: 10
  MOs: 10
  Type: Restricted
  Has overlap: Yes
  Has basis set: Yes
  Valid: Yes
  Has active space: Yes
  Active Orbitals: α=10, β=10
  Has inactive space: No
  Virtual Orbitals: α=0, β=0



### Orbital localization using the QDK MP2 Natural Orbital method

Canonical molecular orbitals from SCF calculations are often delocalized over the entire molecule. Here we use the QDK MP2 Natural Orbital localization method to generate orbitals that tend to yield more compact and chemically meaningful representations.

In [30]:
localizer = create("orbital_localizer", "qdk_mp2_natural_orbitals")
num_orbs = wfn_hf.get_orbitals().get_num_molecular_orbitals()
loc_wfn = localizer.run(wfn_hf, list(range(num_orbs)), list(range(num_orbs)))

## Selecting an active space

Most chemistry applications on quantum computers will require the use of active spaces to focus the quantum calculation on a subset of the electrons and orbitals in the system.

Here, we use `qdk_autocas_eos`, an entropy-based active-space selection method to identify strongly correlated orbitals. This provides an automated approach and returns the selected orbital indices for building the refined active-space Hamiltonian.

Then, we construct the active-space Hamiltonian and compute the multi-configuration wavefunction using the selected active space.

In [31]:
# Run SCI calculation to get correlated information
hamiltonian_constructor = create("hamiltonian_constructor")
loc_orbitals = loc_wfn.get_orbitals()
loc_hamiltonian = hamiltonian_constructor.run(loc_orbitals)

macis_mc = create(
    "multi_configuration_calculator", "macis_asci", calculate_one_rdm=True, calculate_two_rdm=True, ntdets_max=200
    )
_, wfn = macis_mc.run(loc_hamiltonian, *loc_wfn.get_active_num_electrons())

[2025-12-19 21:54:22.800] [ci_solver] [info] [Selected CI Solver]:
[2025-12-19 21:54:22.802] [ci_solver] [info]   NDETS =    100, MATEL_TOL = 2.22045e-16, RES_TOL = 1.00000e-06, MAX_SUB =  200
[2025-12-19 21:54:22.848] [ci_solver] [info]   NNZ   =   2722, H_DUR     = 4.52645e+01 ms
[2025-12-19 21:54:22.848] [ci_solver] [info]   HMEM_LOC = 1.50e-04 GiB
[2025-12-19 21:54:22.848] [ci_solver] [info]   H_SPARSE = 2.72e+01%
[2025-12-19 21:54:22.848] [ci_solver] [info]   * Will generate identity guess
[2025-12-19 21:54:22.848] [davidson] [info] [Davidson Eigensolver]:
[2025-12-19 21:54:22.848] [davidson] [info]   N =    100, MAX_M =  100, RES_TOL = 1.00000e-06
[2025-12-19 21:54:22.857] [davidson] [info] iter =    1, LAM(0) =  -1.466261682624e+00, RNORM =   1.474869700314e-01
[2025-12-19 21:54:22.860] [davidson] [info] iter =    2, LAM(0) =  -1.484804650593e+00, RNORM =   5.538771781871e-02
[2025-12-19 21:54:22.865] [davidson] [info] iter =    3, LAM(0) =  -1.486736478695e+00, RNORM =   1.6929

In [32]:
# Optimize active space using AutoCAS-EOS active space selection

autocas = create("active_space_selector", "qdk_autocas_eos")
autocas_wfn = autocas.run(wfn)
indices, _ = autocas_wfn.get_orbitals().get_active_space_indices()
print(f"AutoCAS-EOS active space: {len(indices)} orbitals, indices={list(indices)}")

AutoCAS-EOS active space: 2 orbitals, indices=[0, 1]


In [33]:
# Build Hamiltonian for the AutoCAS selected active space and calculate CASCI energy

hamiltonian_constructor = create("hamiltonian_constructor")
refined_orbitals = autocas_wfn.get_orbitals()
active_hamiltonian = hamiltonian_constructor.run(refined_orbitals)

mc = create("multi_configuration_calculator", "macis_cas")
e_cas, wfn_cas = mc.run(
        active_hamiltonian, *autocas_wfn.get_active_num_electrons()
)
print(f"Refined CASCI energy: {e_cas:.3f} Hartree")

[2025-12-19 21:55:12.115] [ci_solver] [info] [Selected CI Solver]:
[2025-12-19 21:55:12.115] [ci_solver] [info]   NDETS =      4, MATEL_TOL = 2.22045e-16, RES_TOL = 1.00000e-06, MAX_SUB =  200
[2025-12-19 21:55:12.135] [ci_solver] [info]   NNZ   =     12, H_DUR     = 1.93765e+01 ms
[2025-12-19 21:55:12.135] [ci_solver] [info]   HMEM_LOC = 2.76e-07 GiB
[2025-12-19 21:55:12.135] [ci_solver] [info]   H_SPARSE = 7.50e+01%
[2025-12-19 21:55:12.135] [ci_solver] [info]   * Will generate identity guess
[2025-12-19 21:55:12.135] [davidson] [info] [Davidson Eigensolver]:
[2025-12-19 21:55:12.135] [davidson] [info]   N =      4, MAX_M =    4, RES_TOL = 1.00000e-06
[2025-12-19 21:55:12.150] [davidson] [info] iter =    1, LAM(0) =  -1.476881708445e+00, RNORM =   1.238973940912e-16
[2025-12-19 21:55:12.150] [davidson] [info] Davidson Converged!
[2025-12-19 21:55:12.150] [ci_solver] [info]   DAV_NITER =    1, E0 = -1.476882e+00 Eh, DAVIDSON_DUR = 1.47072e+01 ms
Refined CASCI energy: -1.077 Hartree


In [34]:
from qdk.widgets import MoleculeViewer
from qdk_chemistry.utils.cubegen import generate_cubefiles_from_orbitals

active_orbitals = wfn_cas.get_orbitals()

# Generate cube files for the active orbitals
cube_data = generate_cubefiles_from_orbitals(
    orbitals=active_orbitals,
    grid_size=(40, 40, 40),
    margin=10.0,
    indices=active_orbitals.get_active_space_indices()[0],
    label_maker=lambda p: f"{'occupied' if p < 20 else 'virtual'}_{p + 1:04d}"
)

# Visualize the molecular orbitals together with the structure
MoleculeViewer(molecule_data=structure.to_xyz(), cube_data=cube_data)

## Loading the wavefunction onto a quantum computer

Now that we have calculated the multi-configuration wavefunction for the active space, we can generate a quantum circuit to prepare this state on a quantum computer.

### Identifying the dominant configurations in the wavefunction

In [35]:
import numpy as np
from qdk.widgets import Histogram

from qdk_chemistry.utils.wavefunction import get_top_determinants

# Plot top determinant weights from the CASCI wavefunction
NUM_DETERMINANTS = 4
print(f"Total determinants in the CASCI wavefunction:  {len(wfn_cas.get_active_determinants())}")
print(f"Plotting the top {NUM_DETERMINANTS} determinants by weight.")
top_configurations = get_top_determinants(wfn_cas, max_determinants=NUM_DETERMINANTS)

# pre-sorted 
display(Histogram(bar_values={k.to_string(): np.abs(v)**2 for k, v in top_configurations.items()},))

Total determinants in the CASCI wavefunction:  4
Plotting the top 4 determinants by weight.


### Prepare trial state with two determinants with a given rotation angle

To run quantum phase estimation, we need to prepare an initial trial state that has a good overlap with the true ground state. Here, we construct a trial state using the two most significant determinants from the multi-configuration wavefunction, parameterized by a rotation angle to adjust their contributions.

In [36]:
from utils import prepare_2_dets_trial_state

# Dropping dets
wfn_trial, fidelity = prepare_2_dets_trial_state(
    wfn_cas, rotation_angle=np.pi / 12
)

print(f"Overlap of trial state with sparse wavefunction: {fidelity:.2%}")

top_configurations = get_top_determinants(wfn_trial, max_determinants=2)
display(Histogram(bar_values={k.to_string(): np.abs(v)**2 for k, v in top_configurations.items()},))

Overlap of trial state with sparse wavefunction: 77.27%


### Loading the wavefunction using general state preparation methods

In [37]:
import pandas as pd
from qdk.openqasm import estimate
from qdk.widgets import Circuit

# Generate state preparation circuit for the sparse state using the regular isometry method (Qiskit)
state_prep = create("state_prep", "regular_isometry")
regular_isometry_circuit = state_prep.run(wfn_trial)

# Visualize the regular isometry circuit
display(Circuit(regular_isometry_circuit.get_qsharp()))

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(estimate(regular_isometry_circuit.get_qasm()).logical_counts.items(), columns=['Logical Estimate', 'Counts'])
display(df)

[2025-12-19 21:57:07.109083] [info] [qdk_chemistry.data.circuit] Removing classical qubits will also remove any control operations sourced from them and measurements involving them.


,Logical Estimate,Counts
0,numQubits,4
1,tCount,10
2,rotationCount,11
3,rotationDepth,10
4,cczCount,0
5,ccixCount,0
6,measurementCount,0


### Loading the wavefunction using optimized state preparation methods

In [38]:
# Generate state preparation circuit for the sparse state via sparse isometry (GF2 + X)
state_prep = create("state_prep", "sparse_isometry_gf2x")
sparse_isometry_circuit = state_prep.run(wfn_trial)

# Visualize the sparse isometry circuit
display(Circuit(sparse_isometry_circuit.get_qsharp(remove_classical_qubits=False, remove_idle_qubits=False)))

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(estimate(sparse_isometry_circuit.get_qasm()).logical_counts.items(), columns=['Logical Estimate', 'Counts'])
display(df)

[2025-12-19 21:57:19.888919] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Original matrix rank: 2
[2025-12-19 21:57:19.907950] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Found duplicate row 2 identical to row 0, adding CNOT(0, 2)
[2025-12-19 21:57:19.924265] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Found duplicate row 3 identical to row 1, adding CNOT(1, 3)
[2025-12-19 21:57:19.941483] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Eliminating 2 duplicate rows: [2, 3]
[2025-12-19 21:57:19.954338] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Detected diagonal matrix with rank 2, applying further reduction
[2025-12-19 21:57:19.971249] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Applying diagonal matrix reduction on 2x2 matrix
[2025-12-19 21:57:19.985290] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Applied CNOT(0, 1)
[2025-1

,Logical Estimate,Counts
0,numQubits,4
1,tCount,0
2,rotationCount,1
3,rotationDepth,1
4,cczCount,0
5,ccixCount,0
6,measurementCount,0


## Iterative quantum phase estimation for ground state energy

Kitaev-style iterative quantum phase estimation (IQPE) estimates an eigenphase of the time-evolution operator $U = e^{-iHt}$ using one ancilla qubit and a sequence of controlled-$U^{2^k}$ applications.
 
Each iteration measures one bit of the phase (from most-significant to least-significant) and uses phase feedback to refine the estimate. In this notebook, we apply IQPE to the **qubit-mapped molecular Hamiltonian** with the prepared **trial state** to obtain the ground-state energy from the measured phase.


### Preparing qubit Hamiltonian

First, we map the classical Hamiltonian for the active space to a qubit Hamiltonian that can be measured on a quantum computer. For this example, we use the Jordan-Wigner transformation.

In [39]:
qubit_mapper = create("qubit_mapper", algorithm_name="qiskit", encoding="jordan-wigner")
qubit_hamiltonian = qubit_mapper.run(active_hamiltonian)
print("Qubit Hamiltonian:\n", qubit_hamiltonian.get_summary())

Qubit Hamiltonian:
 Qubit Hamiltonian
  Number of qubits: 4
  Number of terms: 15



### Visualizing the circuit

The circuit for IQPE consists of initial trial state preparation followed by multiple controlled time-evolution operations. This cell visualizes the one iteration of the IQPE circuit.

In [40]:
from qdk_chemistry.algorithms import IterativePhaseEstimation
from qiskit import qasm3

M_PRECISION = 10  # number of phase bits of precision
T_TIME = 0.1  # evolution time
SHOTS_PER_ITERATION = 3
SIMULATOR_SEED = 42

state_prep_circuit = qasm3.loads(sparse_isometry_circuit.qasm)
iqpe = IterativePhaseEstimation(qubit_hamiltonian, T_TIME)

# Create an IQPE circuit for a specific iteration (e.g., the third-to-last)
circuit = iqpe.create_iteration_circuit(state_prep_circuit, iteration=M_PRECISION-3, total_iterations=M_PRECISION)
circuit_qasm = qasm3.dumps(circuit)
with open("iqpe_circuit.qasm", "w") as f:
    f.write(circuit_qasm)
    

### Run iterative QPE on a quantum simulator multiple times to build energy distribution

Since single-shot phase estimates can fluctuate, we execute the IQPE routine on a simulator multiple independent trials to obtain a distribution of energy estimates rather than a single point value.

In [ ]:
from collections import Counter
from pathlib import Path

from utils import run_iqpe

NUM_TRIALS = 30

results = run_iqpe(
    qubit_hamiltonian,
    state_prep_circuit,
    time=T_TIME,
    precision=M_PRECISION,
    shots=SHOTS_PER_ITERATION,
    seed=SIMULATOR_SEED,
    reference_energy=e_cas - active_hamiltonian.get_core_energy(),
    trials=NUM_TRIALS,
)

Path("results").mkdir(exist_ok=True)
for i, qpe_result in enumerate(results):
    qpe_result.to_json_file(f"results/iqpe_result_{i}.qpe_result.json")

In [41]:
from collections import Counter
from pathlib import Path

from qdk_chemistry.data import QpeResult

results_loaded = []
for json_file in Path("results").glob("*qpe_result.json"):
    result = QpeResult.from_json_file(json_file)
    results_loaded.append(result)

print(f"Total energy (energy from phase estimation + core energy) distribution:")
display(
    Histogram(bar_values={f"{k+active_hamiltonian.get_core_energy():.4f}": v for k, v in Counter([result.resolved_energy for result in results_loaded]).items()})
    )

estimated_energy, _ = Counter([result.resolved_energy for result in results_loaded]).most_common(1)[0]
print(f"Total energy from phase estimation: {estimated_energy + active_hamiltonian.get_core_energy():.6f} Hartree")
print(f"Energy difference with CASCI energy: {abs(estimated_energy + active_hamiltonian.get_core_energy() - e_cas):.3e} Hartree")

Total energy (energy from phase estimation + core energy) distribution:


Total energy from phase estimation: -1.072622 Hartree
Energy difference with CASCI energy: 4.260e-03 Hartree
